In [1]:
# write a prompt to define the input and output of a gpt call - expert gpt in a specific task
from utils import get_openai_client, read_web_content, query_system_expert ,extract_message_choice

/Users/anubhavtomar/2023_projects/chatgpt/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
client = get_openai_client()


In [34]:
bse_markets = 'https://www.bseindia.com/markets.html' #crap
# reddit_thread = 'https://www.reddit.com/r/MachineLearning/comments/1bho0r0/in_2024_which_library_is_best_for_time_series/' #blocked
medium_article = 'https://medium.com/@goldengoat/how-to-perform-anomaly-detection-in-time-series-data-with-python-methods-code-example-e83b9c951a37'
members_medium_article = 'https://medium.com/@pspddo/new-python-tools-that-deserve-your-attention-02df2fb987a8'
# content = read_web_content(medium_article)

# content.__dict__

In [4]:
## lets write an agent that can parse the response from the jina reader api. It is builtl for medium type articles.ll
# ideally the medium article should return well structured markdown from reader api -> infer that structure -> return as json 
# classify the story as members or free based on the content recvd -> if its not full article then classify as members only and its a failure for story collecting agent.


In [5]:
system_response_type = {
    "meta": {
        "status": "success" ,
         "message": "if error then ...." # Or "error" if there's an issue
    },
    "data": {
        "article_insights": {
        "title": "...",
        "url": "...",
        "publislhed_time": "...",
        "key_topics": ["...", "...", "..."],
        "summary_200" : "summary in 200 words",

        
        "code_snippets": [
            {
            "language": "...",
            "code": "..."
            },
        ]
        },
        "links": ['list','of','links in the article' ] 
    
        # "cleaned_article": " ... cleanup any special characters or spaces in the article and put under the key. keep full article no summarising"
    }
}

In [36]:
system_prompt_full = f"""Analyze the provided text and extract the available information in the form of structure provided at the end.  
                    FIRST AND  MOST IMPORTANTLY: if the content provided isnt long enough or COMPLETE, or DOESNT LOOK  like a article to you. Update status as failed and message as "MEMBERS_ONLY".
                    meta.status as warning. if the content is too big. (more than 2000 words).
                    Only parse the unstructure data according to the above structure. 
                    if most of the content is not fiiting in the structure of data, update the meta.status to failed, with additional messagelike "NO ARTICLE FOUND".
                    Remember that you are an expert in parsing unstructured markdown content into json, recognising entities very easily.
                    Response structure : {system_response_type.__str__()}.

                        """
system_prompt_chunk = f"""Analyze the provided text and decide wether the article is complete or is cut down? 
                    if the article is members only it would only have some starting part of the actual article, you need to identify it and return
                    FULL_ARTICLE, INCOMPLETE_ARTICLE, UNIDENTIFIED_CONTENT based on the following:
                    UNIDENTIFIED_CONTENT:  when the input content does not look like a BLOG OR AN ARTICLE. but some random website homepage or naviagtions content then mark it as UNIDENTIFIED_CONTENT.

                    Remember that you are an avid reader and recognise how humans read and write documentation and stories. The input is passed by a scraper that has already removed most html and gives you markdown. So use that.
                    FULL_ARTICLE: Only say FULL_ARTICLE when you are 200% sure that the content provided is a BLOG OR ARTICLE like news or educational written by people.
                    INCOMPLETE_ARTICLE: this is a rare case when the provided link is under pay wall. so recognise these paywalls and ONLY WHEN you are sure its incomplete due to the pay-wall, mark as INCOMPLETE.
                    """
system_prompt_full

'Analyze the provided text and extract the available information in the form of structure provided at the end.  \n                    FIRST AND  MOST IMPORTANTLY: if the content provided isnt long enough or COMPLETE, or DOESNT LOOK  like a article to you. Update status as failed and message as "MEMBERS_ONLY".\n                    meta.status as warning. if the content is too big. (more than 2000 words).\n                    Only parse the unstructure data according to the above structure. \n                    if most of the content is not fiiting in the structure of data, update the meta.status to failed, with additional messagelike "NO ARTICLE FOUND".\n                    Remember that you are an expert in parsing unstructured markdown content into json, recognising entities very easily.\n                    Response structure : {\'meta\': {\'status\': \'success\', \'message\': \'if error then ....\'}, \'data\': {\'article_insights\': {\'title\': \'...\', \'url\': \'...\', \'publishe

In [39]:
## read a chosen article - success case
reader = read_web_content(medium_article)
content = reader.content


In [41]:
reader.__dict__

{'_content': b'Title: How to perform anomaly detection in time series data with python? Methods, Code, Example!\n\nURL Source: https://medium.com/@goldengoat/how-to-perform-anomaly-detection-in-time-series-data-with-python-methods-code-example-e83b9c951a37\n\nPublished Time: 2023-06-30T04:11:11.678Z\n\nMarkdown Content:\n[![Image 1: Yu Rong (Tammy) Tsao](https://miro.medium.com/v2/resize:fill:88:88/1*LU6YltLV-yHZXRsTTv9fng@2x.jpeg)](https://medium.com/@goldengoat?source=post_page-----e83b9c951a37--------------------------------)\n\n![Image 2](https://miro.medium.com/v2/resize:fit:700/1*CCLlnUVtuNBo7idKtODghQ.png)\n\nIn this article, we will cover the following topics:\n\n*   Why our business needs anomaly detection?\n*   Types of anomaly and how to choose the anomaly detection algorithms\n*   The brief concept of level shift anomaly detection and isolation forest method\n*   The implementation details in python\n*   A real-world example. Includes the encountered problem of the business

In [12]:
response , client = query_system_expert(system_content=system_prompt_full,
                                        user_input=str(content),max_tokens=1024)
print(response)

ChatCompletion(id='chatcmpl-9WgXlceJubSNApQeXZPZcQw0gb1Lp', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='{\'meta\': {\'status\': \'success\', \'message\': \'Data parsed successfully.\'}, \'data\': {\'article_insights\': {\'title\': \'How to perform anomaly detection in time series data with python? Methods, Code, Example!\', \'url\': \'https://medium.com/@goldengoat/how-to-perform-anomaly-detection-in-time-series-data-with-python-methods-code-example-e83b9c951a37\', \'published_time\': \'2023-06-30T04:11:11.678Z\', \'key_topics\': [\'Why our business needs anomaly detection?\', \'Types of time series anomalies\', \'Anomaly detection algorithms\', \'Implementation example\'], \'summary_200\': "In this article, the importance of anomaly detection in time series data for businesses is discussed. The types of anomalies, including Level Shift Anomalies and Collective Anomalies, are explained. Two anomaly detection algorithms, Level Shi

In [13]:
msg = extract_message_choice(response)
msg

message rcvd:
{'id': 'chatcmpl-9WgXlceJubSNApQeXZPZcQw0gb1Lp', 'choices': [Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='{\'meta\': {\'status\': \'success\', \'message\': \'Data parsed successfully.\'}, \'data\': {\'article_insights\': {\'title\': \'How to perform anomaly detection in time series data with python? Methods, Code, Example!\', \'url\': \'https://medium.com/@goldengoat/how-to-perform-anomaly-detection-in-time-series-data-with-python-methods-code-example-e83b9c951a37\', \'published_time\': \'2023-06-30T04:11:11.678Z\', \'key_topics\': [\'Why our business needs anomaly detection?\', \'Types of time series anomalies\', \'Anomaly detection algorithms\', \'Implementation example\'], \'summary_200\': "In this article, the importance of anomaly detection in time series data for businesses is discussed. The types of anomalies, including Level Shift Anomalies and Collective Anomalies, are explained. Two anomaly detection algorithms, Lev

'{\'meta\': {\'status\': \'success\', \'message\': \'Data parsed successfully.\'}, \'data\': {\'article_insights\': {\'title\': \'How to perform anomaly detection in time series data with python? Methods, Code, Example!\', \'url\': \'https://medium.com/@goldengoat/how-to-perform-anomaly-detection-in-time-series-data-with-python-methods-code-example-e83b9c951a37\', \'published_time\': \'2023-06-30T04:11:11.678Z\', \'key_topics\': [\'Why our business needs anomaly detection?\', \'Types of time series anomalies\', \'Anomaly detection algorithms\', \'Implementation example\'], \'summary_200\': "In this article, the importance of anomaly detection in time series data for businesses is discussed. The types of anomalies, including Level Shift Anomalies and Collective Anomalies, are explained. Two anomaly detection algorithms, Level Shift Anomaly Detection using ADTK and Isolation Forest, are detailed along with their advantages and disadvantages. An implementation example is provided, showcas

In [28]:
msg

"{'meta': {'status': 'success', 'message': 'Data parsed successfully.'}, 'data': {'article_insights': {'title': 'How to perform anomaly detection in time series data with python? Methods, Code, Example!', 'url': 'https://medium.com/@goldengoat/how-to-perform"

In [14]:
# try to ask follow up 
from constants import *
user_input = "expand on the key topics you just mentioned"
new_response = client.chat.completions.create(model=DEFAULT_MODEL,
                                messages=[
                                        {
                                            "role": "user",
                                            "content":user_input
                                        }],
                                temperature=DEFAULT_TEMPERATURE,
                                max_tokens=1024,
                                top_p=DEFAULT_TOP_P
                            )
fmsg = extract_message_choice(new_response)
fmsg
## client cant be treated like a state store. openai  treats each request independently. so to achieve the above req
## store the history on our end, update the messages key in session for each expert. append the user/assistant response as they pile up
## build mechanisms based on speciific use to optimise the context

message rcvd:
{'id': 'chatcmpl-9WgiZBv2hfwaESb7wpuB5d8nsOduF', 'choices': [Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="Sure! Let's expand on the key topics I mentioned earlier:\n\n1. Sustainability: Sustainability refers to the practice of using resources in a way that meets current needs without compromising the ability of future generations to meet their own needs. This includes reducing waste, conserving energy, promoting renewable resources, and protecting ecosystems. Businesses are increasingly adopting sustainable practices as consumers become more environmentally conscious and demand eco-friendly products and services.\n\n2. Diversity and Inclusion: Diversity and inclusion are essential for creating a more equitable and inclusive society. Diversity refers to the presence of different types of people in a group or organization, such as people of different races, genders, ages, and backgrounds. Inclusion, on the other hand, involves 

"Sure! Let's expand on the key topics I mentioned earlier:\n\n1. Sustainability: Sustainability refers to the practice of using resources in a way that meets current needs without compromising the ability of future generations to meet their own needs. This includes reducing waste, conserving energy, promoting renewable resources, and protecting ecosystems. Businesses are increasingly adopting sustainable practices as consumers become more environmentally conscious and demand eco-friendly products and services.\n\n2. Diversity and Inclusion: Diversity and inclusion are essential for creating a more equitable and inclusive society. Diversity refers to the presence of different types of people in a group or organization, such as people of different races, genders, ages, and backgrounds. Inclusion, on the other hand, involves creating a culture where all individuals feel respected, valued, and included. Promoting diversity and inclusion in the workplace can lead to greater innovation, crea

In [ ]:
## done. takeaways ? 
## sub divide this agent to extract smaller things , 
    #  -> one prompt/agent for extract clean content & story meta data (title, time author etc)
    # -> another to extract summary in 200 words, 100 words, 500 words, etc 
    # -> extract key topics (5-10), -
    # -> extract link / images 
## keep api structure uniform with meta for each agent to pass messages related to the content
## reader api is pretty good at parsing article like content -> would be interesting to know what percentage of web content is in form of article?
    # what percentage of "NEW" content is articles ? 
    # who are the top 100 contributors to the web in terms of internet agencies -> what percentage of those are primarily in article form? 
    # ( remove platforms  that have ugc, focus on expert driven content--event if its internet expert ) 
## count tokens before sending the request to openapi (use tik token? )
## do this in gscript to add buttons to google sheet/google docs. create agent in app scripts.

In [7]:
# one last thing , read members only article 
members_medium_article
reader = read_web_content(members_medium_article)
members_content = reader.content

In [25]:
# classify article using a chunk of agent functionality prompt 
response , client = query_system_expert(system_content=system_prompt_chunk,
                                        user_input=str(members_content),max_tokens=64)
msg = extract_message_choice(response)
msg

{'completion_tokens': 4, 'prompt_tokens': 572, 'total_tokens': 576}


'INCOMPLETE_ARTICLE'

In [38]:
# more test links
openai_li = "https://platform.openai.com/docs/examples"
rnd = "https://github.com/BLarzalere/LSTM-Autoencoder-for-Anomaly-Detection/tree/masters"
reader = read_web_content(rnd)
x_content = reader.content
response , client = query_system_expert(system_content=system_prompt_chunk,
                                        user_input=str(x_content),max_tokens=64)
print(response.__dict__)
msg = extract_message_choice(response)
msg

{'id': 'chatcmpl-9WhI8O2Iu0xztAGuSxjdnDWI4WiEJ', 'choices': [Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='UNIDENTIFIED_CONTENT', role='assistant', function_call=None, tool_calls=None))], 'created': 1717579936, 'model': 'gpt-3.5-turbo-0125', 'object': 'chat.completion', 'system_fingerprint': None, 'usage': CompletionUsage(completion_tokens=4, prompt_tokens=665, total_tokens=669)}
{'completion_tokens': 4, 'prompt_tokens': 665, 'total_tokens': 669}


'UNIDENTIFIED_CONTENT'

In [29]:
reader.__dict__

{'_content': b"Title: OpenAI Platform\n\nURL Source: https://platform.openai.com/docs/examples\n\nMarkdown Content:\nIntroducing GPT-4o: our fastest and most affordable flagship model\n\nPrompt examples\n---------------\n\nExplore what's possible with some example prompts\n\nGrammar correction\n\nConvert ungrammatical statements into standard English.\n\nSummarize for a 2nd grader\n\nSimplify text to a level appropriate for a second-grade student.\n\nParse unstructured data\n\nCreate tables from unstructured text.\n\nEmoji Translation\n\nTranslate regular text into emoji text.\n\nCalculate time complexity\n\nFind the time complexity of a function.\n\nExplain code\n\nExplain a complicated piece of code.\n\nKeywords\n\nExtract keywords from a block of text.\n\nProduct name generator\n\nGenerate product names from a description and seed words.\n\nPython bug fixer\n\nFind and fix bugs in source code.\n\nSpreadsheet creator\n\nCreate spreadsheets of various kinds of data.\n\nTweet classifie